# Pooled Motif Run Analysis
### Nathan Alam 6/15/20

Ran on the following genomes:

In [1]:
import os
import json
import sqlite3
import re
import yaml

In [2]:
config = {}
with open("config.yaml") as file:
    config = yaml.load(file)

/Users/nalam/Desktop/genomeminingdemo/lassomining/lib/python3.7/site-packages/ipykernel_launcher.py:3: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  This is separate from the ipykernel package so we can avoid doing imports until


In [3]:
config["outputLogs"] + config["runName"] + ".json"

'/Users/nalam/Desktop/genomeminingdemo/output/runs/nameOfRun.json'

In [4]:
theStuff = ""
with open(config["outputLogs"] + config["runName"] + ".json") as file:
    theStuff = json.load(file)

In [5]:
print("Total peptides found: " + str(theStuff["peptides"]))
print("Pattern used: " + str(theStuff["pattern"]))
print("Cutoff rank: " + str(theStuff["cutoff"]))
print("Time elapsed: " + str(theStuff["totalTime"] / 3600) + " hours")
print("Genomes mined: " + str(len(theStuff["input"])))
print("Average time per genome: " + str(theStuff["totalTime"] / len(theStuff["input"])) + " seconds")

Total peptides found: 1444
Pattern used: M[A-Z]{15,45}T[A-Z][A-Z]{6,8}[DE][A-Z]{5,30}\*
Cutoff rank: 0
Time elapsed: 0.0009823050101598104 hours
Genomes mined: 2
Average time per genome: 1.7681490182876587 seconds


In [6]:
# regular expression function for regular expression search
def regexp(expr, item):
    reg = re.compile(expr)
    return reg.search(item) is not None

In [7]:
conn = sqlite3.connect(config["database"])
conn.create_function("REGEXP", 2, regexp)
c = conn.cursor()

selectionStringGenomes = "SELECT DISTINCT genome FROM lassopeptides WHERE runname is '" + config["runName"] + "'"
distinctGenomes = []
for row in c.execute(selectionStringGenomes):
    distinctGenomes.append(row[0])

c.close()

In [8]:
print("Number of genomes with lasso peptides: " + str(len(distinctGenomes)))

Number of genomes with lasso peptides: 1


In [9]:
conn = sqlite3.connect(config["database"])
conn.create_function("REGEXP", 2, regexp)
c = conn.cursor()

selectionStringGenomes = "SELECT DISTINCT sequence, rank, genome FROM lassopeptides WHERE runname is '" + config["runName"] + "'"
peptides = []
for row in c.execute(selectionStringGenomes):
    peptides.append({
        "sequence": row[0],
        "rank": row[1],
        "genome": row[2],
    })
c.close()

In [10]:
print("DISTINCT lasso peptide hits: " + str(len(peptides)))

DISTINCT lasso peptide hits: 1404


In [11]:
genomeDict = {}
genomeArr = []
for genome in distinctGenomes:
    peptideArr = []
    runningSum = 0
    for peptide in peptides:
        if(peptide["genome"] == genome):
            peptideArr.append(peptide)
            runningSum += peptide["rank"]
    genomeArr.append({
            "genome": genome,
            "average" : (1.0 * runningSum) / len(peptideArr),
            "count": len(peptideArr)
        }
    )
    genomeDict.update({
        genome: peptideArr
    })

In [12]:
genomeArr.sort(reverse=True, key=lambda i: i['average'])

In [13]:
for gen in genomeDict.keys():
    genomeDict[gen].sort(reverse=True, key=lambda i: i["rank"])

In [14]:
for gen in genomeDict.keys():
    rankList = []
    peptideList = []
    for peptide in genomeDict[gen]:
        rankList.append(peptide["rank"])
        peptideList.append(peptide["sequence"])
    genomeDict.update({
        gen: {
            "ranks": rankList,
            "peptides": peptideList
        }
    })

In [15]:
y = genomeDict[genomeArr[0]["genome"]]["ranks"]
x = range(0, len(y))

import matplotlib.pyplot as plt
plt.plot(y)

ModuleNotFoundError: No module named 'matplotlib'

In [16]:
with open("allGenomes.txt", 'w') as file:
    file.write(json.dumps(theStuff["input"]))

In [17]:
with open("genomeHits.txt", 'w') as file:
    file.write(json.dumps(genomeArr))

In [18]:
import pandas as pd

precsv = pd.DataFrame.from_dict(genomeDict)
precsv.to_csv("peptides.csv")